In [1]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 5.3 MB/s 
     |████████████████████████████████| 4.4 MB 66.9 MB/s 
     |████████████████████████████████| 1.8 MB 33.3 MB/s 
     |████████████████████████████████| 9.1 MB 55.6 MB/s 
     |████████████████████████████████| 1.2 MB 61.8 MB/s 
     |████████████████████████████████| 6.6 MB 13.6 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 362 kB 66.1 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
     |████████████████████████████████| 596 kB 55.5 MB/s 
     |████████████████████████████████| 145 kB 55.3 MB/s 
     |████████████████████████████████| 181 kB 55.2 MB/s 
     |████████████████████████████████| 63 kB 555 kB/s 
     |████████████████████████████████| 212 kB 53.4 MB/s 
     |████████████████████████████████| 140 kB 50.4 MB/s 
     |██████████████████████████████

This is the original try for code, now not really clear what it is doing

From here the code should work

In [1]:
# Import libraries

import os
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import matplotlib.pyplot as plt 
import seaborn as sn 




In [2]:
#below is for uploading files to the colab working file 
from google.colab import files
uploaded_files = files.upload()

Saving olid-training-pp-labels.txt to olid-training-pp-labels.txt
Saving olid-eval-preprocessed.tsv to olid-eval-preprocessed.tsv


OLID 

In [3]:
#paths to preprocessed files in drive to OLID
inputfile_eval_a = "a_to_star.tsv"
inputfile_eval_all_caps = "all_caps.tsv"
inputfile_eval_i = "i_to_excl.tsv"
inputfile_eval_punct = "punctuation.tsv"
inputfile_eval_typos = "typos.tsv"
inputfile_eval_u = "u_to_star.tsv"
inputfile_train = "olid-training-pp-labels.txt"
inputfile_eval = "olid-eval-preprocessed.tsv"
#make inputfiles for eval

In [4]:
#load the OLID data into pandas dataframe and specify column names
train_df = pd.read_csv(inputfile_train, sep="\t", header=0, names = ['ID', 'text', 'labels'])
train_df = train_df[['text', 'labels']]
eval_df = pd.read_csv(inputfile_eval, sep="\t", header=0, names = ['ID', 'text', 'labels'])
eval_df = eval_df[['text', 'labels']]
#train_df.columns = ['text','labels']
eval_df.columns = ['text','labels']

In [5]:
#statistics OLID and inspection

#print(train_df.head(10))
print(eval_df.head(20))
train_df = train_df[:5852]
print("train:", train_df.groupby('labels').count())
print("total train:", len(train_df))

print("test:", eval_df.groupby('labels').count())
print("total test:", len(eval_df))


                                                 text  labels
0   #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...       1
1   #ConstitutionDay is revered by Conservatives, ...       0
2   #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...       0
3   #Watching #Boomer getting the news that she is...       0
4   #NoPasaran: Unity demo to oppose the far-right...       1
5            . . . What the fuck did he do this time?       1
6   #RAP is a form of ART! Used to express yoursel...       0
7   @USER Do you get the feeling he is kissing @US...       1
8   5 Tips to Enhance Audience Connection on Faceb...       0
9   #BiggBossTamil janani won the task. She is goi...       0
10  #Conservatives - the party of low taxation 😂 #...       0
11  𝓒-𝓒𝓮𝓭𝓻𝓪𝓷..  The Nord cannot make a single move...       0
12    #ConsTOO THE PLACE FOR FED UP CONSERVATIVES !!!       0
13  #GreatAwakening #QAnon #PatriotsUnited #WWG1WG...       0
14                        @USER Nigga ware da hits at       1
15  #Sto

HASOC

In [6]:
#paths to files in drive to HASOC
inputfile_hasoc = ('HASOC_SOURCE_english_dataset_train_preprocessed.tsv')
train_df_hasoc = pd.read_csv(inputfile_hasoc, sep="\t", header=0, names = ['ID', 'text', 'labels'])
train_df_hasoc = train_df_hasoc[['text', 'labels']]

#inputfile_hasoc_test = "HASOC_SOURCE_english_dataset_test_preprocessed.tsv"
#eval_df_hasoc = pd.read_csv(inputfile_hasoc_test, sep="\t", header=0, names = ['ID', 'text', 'labels'])
#eval_df_hasoc = eval_df_hasoc[['text', 'labels']]



In [152]:
#statistics HASOC and inspection

print(train_df_hasoc.head(5))
#print(eval_df_hasoc.head(5))

print("train:", train_df_hasoc.groupby('labels').count())
print("total train:", len(train_df_hasoc))

#print("test:", eval_df_hasoc.groupby('labels').count())
#print("total test:", len(eval_df_hasoc))


                                                text  labels
0  #DhoniKeepsTheGlove | WATCH: Sports Minister K...       0
1  @politico No. We should remember very clearly ...       1
2  @cricketworldcup Guess who would be the winner...       0
3  Corbyn is too politically intellectual for #Bo...       0
4  All the best to #TeamIndia for another swimmin...       0
train:         text
labels      
0       3591
1       2261
total train: 5852


Together dataset (concatination)

Here different model args can be used, check first code block at the top 


Load model and possible parameters

In [6]:
model = ClassificationModel('bert', 'bert-base-cased', args={'num_train_epochs': 4, 'reprocess_input_data': True, 'overwrite_output_dir': True}, use_cuda=True)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

OLID model train

In [7]:
olid_model = model.train_model(train_df)

  0%|          | 0/5852 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/732 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/732 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/732 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/732 [00:00<?, ?it/s]

HASOC model train

In [8]:
hasoc_model = model.train_model(train_df_hasoc)

  0%|          | 0/5852 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/732 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/732 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/732 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/732 [00:00<?, ?it/s]

Together model

**Evaluation**
make sure to change input of evaluation and model names

In [8]:
predicted, probabilities = model.predict(eval_df.text.to_list())


  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [9]:
eval_df['predicted'] = predicted

In [10]:
# Result 
print(classification_report(eval_df['labels'], eval_df['predicted']))

from google.colab import files

eval_df.to_csv('in_domain_OLID_test_predicted.csv') #make sure to change outputfile names
files.download('in_domain_OLID_test_predicted.csv') #make sure to change outputfile names



              precision    recall  f1-score   support

           0       0.90      0.84      0.87       620
           1       0.65      0.76      0.70       240

    accuracy                           0.82       860
   macro avg       0.77      0.80      0.78       860
weighted avg       0.83      0.82      0.82       860



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tries with Checklist

In [ ]:

import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR

In [ ]:
import numpy as np
predicted, probabilities = model.predict([eval_df_HASOC])

print(predicted[0], probabilities[0])

model = ClassificationModel('bert', 'bert-base-cased', args={'num_train_epochs': 4, 'reprocess_input_data': True, 'overwrite_output_dir': True}, use_cuda=True)0-
def predict_proba(inputs):
    p1 = np.array([(model.predict(x)[0] + 1)/2. for x in inputs]).reshape(1, -1)
    print("hello", p1)
    p0 = 1- p1
    return np.hstack((p0, p1))
predict_proba([[df_5_3_to_reset]])


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 [ 1.484375   -1.10644531]


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

hello [[0.5]]


array([[0.5, 0.5]])

In [ ]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
data = eval_df["text"]
def batch_predict(model, data, batch_size=128):
    ret = []
    for d in chunks(data, batch_size):
        ret.extend(model(d))
    return ret

batch_predict(model, data)
"""
import numpy as np
def pred_and_conf(data):
    # change format to softmax, make everything in [0.33, 0.66] range be predicted as neutral
    preds = batch_predict(model, data)
    pr = np.array([x['score'] if x['label'] == '1' else 1 - x['score'] for x in preds])
    pp = np.zeros((pr.shape[0], 3))
    
    neg = pr < 0.5 - mn
    pp[neg, 0] = 1 - pr[neg]
    pp[neg, 2] = pr[neg]
    pos = pr > 0.5 + mn
    pp[pos, 0] = 1 - pr[pos]
    pp[pos, 2] = pr[pos]
    neutral_pos = (pr >= 0.5) * (pr < 0.5 + mn)
    pp[neutral_pos, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_pos] - 0.5)
    pp[neutral_pos, 2] = 1 - pp[neutral_pos, 1]
    neutral_neg = (pr < 0.5) * (pr > 0.5 - mn)
    pp[neutral_neg, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_neg] - 0.5)
    pp[neutral_neg, 0] = 1 - pp[neutral_neg, 1]
    preds = np.argmax(pp, axis=1)
    return preds, pp
  """

TypeError: ignored

In [ ]:
from checklist.pred_wrapper import PredictorWrapper
wrapped_pp = PredictorWrapper.wrap_softmax(predict_proba)

In [ ]:
wrapped_pp(['good'])

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(array([0, 0, 0, 1]),
 array([[0.5, 0.5],
        [0.5, 0.5],
        [0.5, 0.5],
        [0. , 1. ]]))

**WandB try-outs**

In [ ]:
#try for splitting with artifacts
run = wandb.init(job_type='train')
artifact = wandb.Artifact('HASOC-dataset', type='dataset')

# Add a file to the artifact's contents
artifact.add_file('HASOC_english_dataset_train_preprocessed.tsv')

# Save the artifact version to W&B and mark it as the output of this run
run.log_artifact(artifact)

# Query W&B for an artifact and mark it as input to this run
artifact = run.use_artifact('bike-dataset:v0')

# Download the artifact's contents
artifact_dir = artifact.download()
